In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of stocks
stocks = ['GOOGL', 'AMZN', 'AAPL', 'META', 'MSFT', 'NVDA', 'BABA', 'WMT', 'TSLA', 'DOCU', 'PANW', 'UNH', 'MSTR', 'ZM', 'MA', 'V']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  17 of 17 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# rename tickers to more readable names, ^DJI -> DJ, ^GSPC -> SP, ^IXIC -> NQ
df.columns = pd.MultiIndex.from_tuples([
    (
        col[0].replace('^DJI', 'DJ').replace('^GSPC', 'SP').replace('^IXIC', 'NQ').replace('^GDAXI', 'DAX').replace('^N225', 'NKY').replace('RTY=F', 'RUT'),
        col[1]
    ) for col in df.columns
])


# Show the result
df.head()

ZM                                                   EBAY  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2020-01-02  68.800003  69.235001  68.019997  68.720001  1315500.0  33.009012   
2020-01-03  67.620003  68.680000  67.099998  67.279999  1127900.0  32.564779   
2020-01-06  66.629997  70.495003  65.811996  70.320000  3151600.0  32.437866   
2020-01-07  70.290001  72.838997  69.370003  71.900002  6985400.0  32.338137   
2020-01-08  71.809998  73.290001  71.029999  72.550003  2482300.0  32.374403   

                                                        ...         UNH  \
                 High        Low      Close     Volume  ...        Open   
Date                                                    ...               
2020-01-02  33.090606  32.764232  32.909286  4979300.0  ...  268.377516   
2020-01-03  32.727966  32.437857  32.601044  3960900.0  ...  262.251924   
2020-01-06  32.474131  32.147757  32.437866  5866000.0  ...  262.918297   
2020-01-07  32.365338  32.048028  32.292809  6867400.0  ...  265.383181   
2020-01-08  32.709845  32.247481  32.274677  6072600.0  ...  266.113478   

                                                               GOOGL  \
                  High         Low       Close     Volume       Open   
Date                                                                   
2020-01-02  269.947723  264.552417  267.026398  2543400.0  66.958671   
2020-01-03  266.460458  259.595349  264.324249  2711400.0  66.938316   
2020-01-06  266.259553  262.206227  266.159119  3079100.0  67.118569   
2020-01-07  266.104361  262.534876  264.552429  2492900.0  69.543349   
2020-01-08  271.481397  264.780636  270.130280  3379200.0  69.263289   

                                                         
                 High        Low      Close      Volume  
Date                                                     
2020-01-02  67.965225  66.863328  67.965225  27278000.0  
2020-01-03  68.216995  66.904545  67.609680  23408000.0  
2020-01-06  69.437081  67.087291  69.411758  46768000.0  
2020-01-07  69.694308  69.101397  69.277679  34330000.0  
2020-01-08  70.108958  69.154542  69.770790  35314000.0  

[5 rows x 85 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()